# Data Engineering Technical test

Welcome to the techical test for Data Engineering. The setup of the test attempts to give guide rails for completing the exercise whilst leaving some appropriate room to demonstrate your POV on best practice and problem solving. If you wish to use other python libraries, than that is fine.

## Introduction
For this technical test, we are going to use open source airbnb data for amsterdam available at the following URL:
http://insideairbnb.com/get-the-data.html

Over 3 parts, the test is intended to 'simulate' a brief end to end data engineering exercise, where we will test your data wrangling skills in both Python and SQL, to demonstrate your capability in those technologies. As the test is mean't simulate a real task, clearly you may utilise resources across the web to help you solve these tasks as you would day to day. 

Some that you may want to reference are below:

https://pandas.pydata.org/docs/

https://www.sqlite.org/lang.html

## Part 1: Python - Reading and exploring raw datasets
In this section of the test we will be assessing your python skills. You should provide your coded, executable answers between the markers `#** Write your code here **#`

In [ ]:
# Perform your python imports here
import pandas as pd
import sqlite3 as sql

In [ ]:
# Use python or bash to download, unzip and read files from the following locations
# http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/listings.csv.gz
# http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/calendar.csv.gz
# http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/reviews.csv.gz

# There are a number of ways of doing this, so only constraint is user is to end up with 3 datastructures 
# (we would recommend dataframes) that contain data from the three source files.

#** Write your code here **#

df_calendar = pd.read_csv("http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/calendar.csv.gz")
df_listings = pd.read_csv("http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/listings.csv.gz")
df_reviews = pd.read_csv("http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2021-04-09/data/reviews.csv.gz")

#** Write your code here **#

# Assessing and exploring the data
Next we will perform a few common tasks on the data whilst it's still in python memory. Let's focus on the listings data in `df_listings`.

Assess the following:
- list metadata about the columns; their names and types.
- generate a pandas style summary for the dataset.
- count the number of rows in the dataframe.
- find the number of missing values across the dataframe (NaN in numeric arrays, None or NaN in object arrays, NaT in datetime ).
- assess which columns contain the missing values.
- show the unique values in the column

***The jupyter truncation of large answers is fine and candidates are not expected to set config to show every entry in answer. We are more interested in your approach than answer.***

In [ ]:
# list metadata about the columns; their names and types.
#** Write your code here **#
df_listings.info()

#** Write your code here **#

In [ ]:
# generate a summary for the dataset.
#** Write your code here **#
df_listings.describe()

#** Write your code here **#

In [ ]:
# count the number of rows in the dataframe.
#** Write your code here **#

len(df_listings)

# or df_listings.shape[0]

#** Write your code here **#

In [ ]:
# find the total number of missing values across the dataframe.
#** Write your code here **#
df_listings.isnull().sum().sum()

#** Write your code here **#

In [ ]:
# assess which columns contain the missing values.
#** Write your code here **#
list(df_listings.columns[df_listings.isnull().any()])

#** Write your code here **#

In [ ]:
# show the unique values in the column host_name
#** Write your code here **#
list(df_listings["host_name"].unique())

#** Write your code here **#
# repeat this command two cells below

In [ ]:
# change all instances of the name Daniel for Danny and Alexander for Alex in the column host_name
#** Write your code here **#
df_listings["host_name"] = df_listings["host_name"].str.replace("Daniel","Danny", regex=False).str.replace("Alexander","Alex", regex=False)


#** Write your code here **#

In [ ]:
# Now repeat the command you used two cells above this to show the change has taken place in column host_name.
#** Write your code here **#
list(df_listings["host_name"].unique())

#** Write your code here **#

# Part 2. File formats
In this section we briefly assess your understanding of modern data formats.

In [ ]:
# convert df_calendar data to parquet file named df_calendar.parquet 
#** Write your code here **#
df_calendar.to_parquet('df_calendar.parquet')

#** Write your code here **#
# Let's compare sizes of original csv and the parquet file.
df_calendar.to_csv('df_calendar.csv')
!ls -lh *calendar*

Briefly explain why there is likely a difference between the sizes of CSV and parquet in the box below.

### Why is the Parquet file Smaller?

Parquet is a columnar format which also uses compression.  Given that similar values tend to reside within a column rather than across a row, then dictionary type compression algorithms (such as GZip) work well against a columnar format like Parquet and produce significant reductions in file size.


# Part 3. Ingesting data into a database
In this section of the test we are going to explore your SQL skills.

In [ ]:
# first lets release memory from the previous python work and unzip and load our preprepared SQL lite database
try:
    del(df_calendar)
    del(df_listings)
    del(df_reviews)
except NameError: pass

!gunzip opensource_airbnb.db.gz
conn = sql.connect('opensource_airbnb.db')

We will use the Jupyter SQL magic to enable you to write vanilla SQL with little python wrap. https://github.com/catherinedevlin/ipython-sql. There is an example below.

In [ ]:
# first load the sql magic extension.
%load_ext sql

Below we show a simple example of multiline sql with sql magic, where the first line connects to the database you've loaded data into and runs a basic query

In [ ]:
%%sql sqlite:///opensource_airbnb.db
    SELECT DISTINCT UPPER(property_type) as property_type 
    FROM listings 
    limit 5

## Exercises 
Now we would like you to write SQL to solve the two following problems

**Problem 1** 

Output the Top 15 listings that accomodates 5-8 people and has the property type 'Boat', ordered by review_scores_cleanliness and has bucketed prices from '\\$0 - \\$100', '\\$100 - \\$200', '\\$200 - \\$300', '\\$300 - \\$400', '\\$400 - \\$500', '\\$500+'. Your result should include `id`, `listing_url`, `review_scores_cleanliness` and your `bucketed_price`. Use the `price` column for bucketing and use the following snippet to enable you to treat this column a numerical should you wish - `CAST(substr(price, 2) as REAL) as price_for_bucketing`.

In [ ]:
%%sql sqlite:///opensource_airbnb.db

/*

(1) Note that this query currently doesn't return 15 listings since there are only 12 listings 
    which are boats that accommodate 5-8 people
(2) If more than 15 listings existed, this query could return more than 15 rows since ties are included
    in the RANK() function.  At an extreme, if they all had the same cleanliness score then RANK() would 
    equal one for all records and they would all be returned.
    
*/

WITH RankedFilteredData AS (
    SELECT
        id,
        listing_url,
        review_scores_cleanliness,
        RANK() OVER (ORDER BY review_scores_cleanliness DESC) AS cleanliness_ranking,
        CAST(substr(price, 2) as REAL) as price_for_bucketing
    FROM listings
    WHERE 
        accommodates BETWEEN 5 AND 8
    AND property_type = 'Boat'
)
SELECT
    id,
    listing_url,
    review_scores_cleanliness,
    CASE
        WHEN price_for_bucketing >=0   AND price_for_bucketing < 100 THEN '$0 - $100'
        WHEN price_for_bucketing >=100 AND price_for_bucketing < 200 THEN '$100 - $200'
        WHEN price_for_bucketing >=200 AND price_for_bucketing < 300 THEN '$200 - $300'
        WHEN price_for_bucketing >=300 AND price_for_bucketing < 400 THEN '$300 - $400'
        WHEN price_for_bucketing >=400 AND price_for_bucketing < 500 THEN '$400 - $500'
        WHEN price_for_bucketing >=500 THEN '$500+'
        ELSE 'Invalid Price'
    END AS bucketed_price
FROM RankedFilteredData
WHERE
    cleanliness_ranking <= 15
    

**Problem 2** 

Find the `listing_id`, `host_name` as well as earliest available `date` and `price` of that date for the most reviewed listing in the data. That is the listing that has received the most reviews. It should return only one row.

In [ ]:
%%sql sqlite:///opensource_airbnb.db

WITH listings_and_review_counts AS (
    SELECT
        l.id AS listing_id,
        l.host_name,
        COUNT(r.id) AS review_count
    FROM 
        listings l
        INNER JOIN reviews r
            ON l.id = r.listing_id
    GROUP BY
        l.id,
        l.host_name
), listings_and_ranked_review_counts AS (
    SELECT
        listing_id,
        host_name,
        review_count,
        RANK() OVER (ORDER BY review_count DESC) AS review_count_rank
    FROM 
        listings_and_review_counts   
), best_viewed_listing AS (
    SELECT
        listing_id,
        host_name,
        review_count
    FROM 
        listings_and_ranked_review_counts
    WHERE review_count_rank = 1
), best_viewed_listing_with_earliest_date AS (
    SELECT
        l.listing_id,
        l.host_name,
        MIN(c.date) AS earliest_date
    FROM 
        best_viewed_listing l
        INNER JOIN calendar c
            ON l.listing_id = c.listing_id
    GROUP BY
        l.listing_id,
        l.host_name
)
SELECT
    b.listing_id,
    b.host_name,
    b.earliest_date,
    c.price
FROM
    best_viewed_listing_with_earliest_date b
    INNER JOIN calendar c
        ON b.earliest_date = c.date AND
           b.listing_id = c.listing_id
